### 0. 법정동코드(행정동코드)

In [20]:
import os
print(os.getcwd())  

c:\HIuniv_Project-hyeonseo\notebooks


In [22]:
# 행정구역별 코드를 구하기 위해 국토교통부 법정동 코드 csv 파일을 불러온다.
import pandas as pd
csv_path = r'../data/raw/국토교통부_전국 법정동_20250415.csv'
cd = pd.read_csv(csv_path)
cd.head()

,법정동코드,시도명,시군구명,읍면동명,리명,순위,생성일자,삭제일자,과거법정동코드
0,1100000000,서울특별시,NaN,NaN,NaN,11.0,1988-04-23,NaN,NaN
1,1111000000,서울특별시,종로구,NaN,NaN,1.0,1988-04-23,NaN,NaN
2,1111010100,서울특별시,종로구,청운동,NaN,1.0,1988-04-23,NaN,NaN
3,1111010200,서울특별시,종로구,신교동,NaN,2.0,1988-04-23,NaN,NaN
4,1111010300,서울특별시,종로구,궁정동,NaN,3.0,1988-04-23,NaN,NaN


In [23]:
# 데이터 분석에 필요없는 컬럼들은 전부 drop하고, 마찬가지로 읍면동명이 NaN인 행들도 전부 drop한다.
# 단, 여기서는 읍면동에 대한 법정동 코드를 구하기 위해 다 날리지만 시군구명을 사용할시는 다르게 처리해야함.

cd = cd.drop(columns=['리명', '순위', '생성일자', '삭제일자', '과거법정동코드'])
cd = cd.dropna(subset=['읍면동명'])
display(cd.head())

,법정동코드,시도명,시군구명,읍면동명
2,1111010100,서울특별시,종로구,청운동
3,1111010200,서울특별시,종로구,신교동
4,1111010300,서울특별시,종로구,궁정동
5,1111010400,서울특별시,종로구,효자동
6,1111010500,서울특별시,종로구,창성동


In [24]:
# '법정동코드' 컬럼을 기준으로 중복된 행 제거 (첫 번째 행 유지)
cd = cd.drop_duplicates(subset=['법정동코드'], keep='first')

display(cd.head())

,법정동코드,시도명,시군구명,읍면동명
2,1111010100,서울특별시,종로구,청운동
3,1111010200,서울특별시,종로구,신교동
4,1111010300,서울특별시,종로구,궁정동
5,1111010400,서울특별시,종로구,효자동
6,1111010500,서울특별시,종로구,창성동


In [25]:
# '시도명', '시군구명', '읍면동명' 컬럼 조합을 기준으로 중복된 행 제거 (첫 번째 행 유지)
cd = cd.drop_duplicates(subset=['시도명', '시군구명', '읍면동명'], keep='first')

display(cd.head())

,법정동코드,시도명,시군구명,읍면동명
2,1111010100,서울특별시,종로구,청운동
3,1111010200,서울특별시,종로구,신교동
4,1111010300,서울특별시,종로구,궁정동
5,1111010400,서울특별시,종로구,효자동
6,1111010500,서울특별시,종로구,창성동


### 1. 행정구역별 15세 미만 인구수

In [26]:
# 읍면동 별 아동 인구수 데이터를 불러온다.
pop15 = pd.read_excel(r'../data/raw/읍면동_15세미만.xlsx')

c:\Users\Metato\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [27]:
pop15.head()

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),2024
0,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
1,서울특별시,소계,소계,15세미만,807110
2,NaN,종로구,소계,15세미만,9276
3,NaN,NaN,사직동,15세미만,619
4,NaN,NaN,삼청동,15세미만,130


In [28]:
# 첫 번째 행을 컬럼 헤더로 설정
pop15.columns = pop15.iloc[0]

# 첫 번째 행 삭제
pop15 = pop15[1:].reset_index(drop=True)

display(pop15.head())

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
0,서울특별시,소계,소계,15세미만,807110
1,NaN,종로구,소계,15세미만,9276
2,NaN,NaN,사직동,15세미만,619
3,NaN,NaN,삼청동,15세미만,130
4,NaN,NaN,부암동,15세미만,683


In [29]:
# '행정구역별(읍면동)(1)', '행정구역별(읍면동)(2)', '행정구역별(읍면동)(3)' 컬럼의 NaN 값을 앞의 값으로 채우기
pop15['행정구역별(읍면동)(1)'] = pop15['행정구역별(읍면동)(1)'].ffill()
pop15['행정구역별(읍면동)(2)'] = pop15['행정구역별(읍면동)(2)'].ffill()
pop15['행정구역별(읍면동)(3)'] = pop15['행정구역별(읍면동)(3)'].ffill()

display(pop15.head())

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
0,서울특별시,소계,소계,15세미만,807110
1,서울특별시,종로구,소계,15세미만,9276
2,서울특별시,종로구,사직동,15세미만,619
3,서울특별시,종로구,삼청동,15세미만,130
4,서울특별시,종로구,부암동,15세미만,683


In [30]:
# '행정구역별(읍면동)(3)' 컬럼의 값이 '소계'인 행들 삭제
pop15 = pop15[pop15['행정구역별(읍면동)(3)'] != '소계'].reset_index(drop=True)

display(pop15.head())

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
0,서울특별시,종로구,사직동,15세미만,619
1,서울특별시,종로구,삼청동,15세미만,130
2,서울특별시,종로구,부암동,15세미만,683
3,서울특별시,종로구,평창동,15세미만,1484
4,서울특별시,종로구,무악동,15세미만,903


In [31]:
# 컬럼 이름 변경
pop15 = pop15.rename(columns={'행정구역별(읍면동)(1)': '시도명', '행정구역별(읍면동)(2)': '시군구명', '행정구역별(읍면동)(3)': '읍면동명'})

display(pop15.head())

,시도명,시군구명,읍면동명,연령별(1),총인구(명)
0,서울특별시,종로구,사직동,15세미만,619
1,서울특별시,종로구,삼청동,15세미만,130
2,서울특별시,종로구,부암동,15세미만,683
3,서울특별시,종로구,평창동,15세미만,1484
4,서울특별시,종로구,무악동,15세미만,903


In [32]:
# '연령별(1)' 컬럼 삭제
pop15 = pop15.drop(columns=['연령별(1)'])

# '총인구(명)' 컬럼 이름을 '아동인구수'로 변경
pop15 = pop15.rename(columns={'총인구(명)': '아동인구수'})

display(pop15.head())

,시도명,시군구명,읍면동명,아동인구수
0,서울특별시,종로구,사직동,619
1,서울특별시,종로구,삼청동,130
2,서울특별시,종로구,부암동,683
3,서울특별시,종로구,평창동,1484
4,서울특별시,종로구,무악동,903


In [33]:
# '아동인구수' 컬럼의 데이터 타입을 숫자로 변환 (errors='coerce'를 사용하여 변환 불가능한 값은 NaN으로 처리)
pop15['아동인구수'] = pd.to_numeric(pop15['아동인구수'], errors='coerce')

# '아동인구수' 컬럼을 nullable integer 타입으로 변환 (NaN 포함 가능)
pop15['아동인구수'] = pop15['아동인구수'].astype('Int64')

display(pop15.head())

,시도명,시군구명,읍면동명,아동인구수
0,서울특별시,종로구,사직동,619
1,서울특별시,종로구,삼청동,130
2,서울특별시,종로구,부암동,683
3,서울특별시,종로구,평창동,1484
4,서울특별시,종로구,무악동,903


### 2. 행정구역별 65세 이상 인구수


In [36]:
# 읍면동 별 고령자 인구수 데이터를 불러온다.
pop65 = pd.read_excel(r'../data/raw/읍면동_65세이상.xlsx')
pop65.head()

c:\Users\Metato\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),2024
0,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
1,서울특별시,소계,소계,65세이상,1791491
2,NaN,종로구,소계,65세이상,28376
3,NaN,NaN,사직동,65세이상,1826
4,NaN,NaN,삼청동,65세이상,577


In [37]:
# 첫 번째 행을 컬럼 헤더로 설정
pop65.columns = pop65.iloc[0]

# 첫 번째 행 삭제
pop65 = pop65[1:].reset_index(drop=True)

# '행정구역별(읍면동)(1)', '행정구역별(읍면동)(2)', '행정구역별(읍면동)(3)' 컬럼의 NaN 값을 앞의 값으로 채우기
pop65['행정구역별(읍면동)(1)'] = pop65['행정구역별(읍면동)(1)'].ffill()
pop65['행정구역별(읍면동)(2)'] = pop65['행정구역별(읍면동)(2)'].ffill()
pop65['행정구역별(읍면동)(3)'] = pop65['행정구역별(읍면동)(3)'].ffill()

# '행정구역별(읍면동)(3)' 컬럼의 값이 '소계'인 행들 삭제
pop65 = pop65[pop65['행정구역별(읍면동)(3)'] != '소계'].reset_index(drop=True)

# 컬럼 이름 변경
pop65 = pop65.rename(columns={'행정구역별(읍면동)(1)': '시도명', '행정구역별(읍면동)(2)': '시군구명', '행정구역별(읍면동)(3)': '읍면동명'})

# '연령별(1)' 컬럼 삭제
pop65 = pop65.drop(columns=['연령별(1)'])

# '총인구(명)' 컬럼 이름을 '고령자인구수'로 변경
pop65 = pop65.rename(columns={'총인구(명)': '고령자인구수'})

display(pop65.head())

,시도명,시군구명,읍면동명,고령자인구수
0,서울특별시,종로구,사직동,1826
1,서울특별시,종로구,삼청동,577
2,서울특별시,종로구,부암동,1953
3,서울특별시,종로구,평창동,3854
4,서울특별시,종로구,무악동,1492


In [38]:
# '고령자인구수' 컬럼의 데이터 타입을 숫자로 변환 (errors='coerce'를 사용하여 변환 불가능한 값은 NaN으로 처리)
pop65['고령자인구수'] = pd.to_numeric(pop65['고령자인구수'], errors='coerce')

# 필요하다면 NaN 값을 처리 (예: 0으로 채우거나 해당 행 삭제) - 여기서는 NaN을 그대로 두고 nullable integer로 변환
# '고령자인구수' 컬럼을 nullable integer 타입으로 변환 (NaN 포함 가능)
pop65['고령자인구수'] = pop65['고령자인구수'].astype('Int64')

display(pop65.head())

,시도명,시군구명,읍면동명,고령자인구수
0,서울특별시,종로구,사직동,1826
1,서울특별시,종로구,삼청동,577
2,서울특별시,종로구,부암동,1953
3,서울특별시,종로구,평창동,3854
4,서울특별시,종로구,무악동,1492


### 3. 행정구역별 외국인 인구수 (미숙한 한국어는 재난 발생시에 위험을 높힐 수 있기에 점수 계산에 포함)

In [39]:
popFo = pd.read_excel(r'../data/raw/읍면동_총인구수.xlsx')
popFo.head()

c:\Users\Metato\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),2024,2024.1
0,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명),내국인(명)
1,서울특별시,소계,소계,합계,9335444,8968153
2,NaN,종로구,소계,합계,144486,132177
3,NaN,NaN,사직동,합계,8411,8138
4,NaN,NaN,삼청동,합계,2060,1994


In [40]:
# 첫 번째 행을 컬럼 헤더로 설정
popFo.columns = popFo.iloc[0]

# 첫 번째 행 삭제
popFo = popFo[1:].reset_index(drop=True)

# '행정구역별(읍면동)(1)', '행정구역별(읍면동)(2)', '행정구역별(읍면동)(3)' 컬럼의 NaN 값을 앞의 값으로 채우기
popFo['행정구역별(읍면동)(1)'] = popFo['행정구역별(읍면동)(1)'].ffill()
popFo['행정구역별(읍면동)(2)'] = popFo['행정구역별(읍면동)(2)'].ffill()
popFo['행정구역별(읍면동)(3)'] = popFo['행정구역별(읍면동)(3)'].ffill()

In [41]:
# '행정구역별(읍면동)(3)' 컬럼의 값이 '소계'인 행들 삭제
popFo = popFo[popFo['행정구역별(읍면동)(3)'] != '소계'].reset_index(drop=True)

# 컬럼 이름 변경
popFo = popFo.rename(columns={'행정구역별(읍면동)(1)': '시도명', '행정구역별(읍면동)(2)': '시군구명', '행정구역별(읍면동)(3)': '읍면동명'})

# '연령별(1)' 컬럼 삭제
popFo = popFo.drop(columns=['연령별(1)'])

# '총인구(명)' 컬럼 이름을 '고령자인구수'로 변경
popFo = popFo.rename(columns={'총인구(명)': '총인구', '내국인(명)' : '내국인'})

display(popFo.head())

,시도명,시군구명,읍면동명,총인구,내국인
0,서울특별시,종로구,사직동,8411,8138
1,서울특별시,종로구,삼청동,2060,1994
2,서울특별시,종로구,부암동,9057,8712
3,서울특별시,종로구,평창동,16392,16098
4,서울특별시,종로구,무악동,7446,7398


In [42]:
# '총인구'와 '내국인' 컬럼을 숫자로 변환 (errors='coerce'를 사용하여 변환 불가능한 값은 NaN으로 처리)
# '내국인' 컬럼이 존재하는 경우에만 변환
if '내국인' in popFo.columns:
    popFo['내국인'] = pd.to_numeric(popFo['내국인'], errors='coerce')
popFo['총인구'] = pd.to_numeric(popFo['총인구'], errors='coerce')


# '총인구' - '내국인'으로 '외국인' 컬럼 생성
# '내국인' 컬럼이 존재하는 경우에만 뺄셈 수행, 아니면 NaN으로 초기화
if '내국인' in popFo.columns:
    popFo['외국인'] = popFo['총인구'] - popFo['내국인']
else:
    popFo['외국인'] = pd.NA # Or 0, depending on desired behavior when '내국인' is missing


# '총인구'와 '외국인' 컬럼의 NaN 값을 0으로 채우고 정수형으로 변환
popFo['총인구'] = popFo['총인구'].fillna(0).astype(int)
popFo['외국인'] = popFo['외국인'].fillna(0).astype(int)


# '내국인' 컬럼 삭제
if '내국인' in popFo.columns:
    popFo = popFo.drop(columns=['내국인'])

display(popFo.head())

,시도명,시군구명,읍면동명,총인구,외국인
0,서울특별시,종로구,사직동,8411,273
1,서울특별시,종로구,삼청동,2060,66
2,서울특별시,종로구,부암동,9057,345
3,서울특별시,종로구,평창동,16392,294
4,서울특별시,종로구,무악동,7446,48


### 4. 행정구역별 장애인 인구수

In [43]:
#

### 5. 전체 데이터들 merge

In [44]:
total = pd.merge(pop15, pop65, on=['시도명', '시군구명', '읍면동명'], how='inner')
total = pd.merge(total, popFo, on=['시도명', '시군구명', '읍면동명'], how='inner')

display(total.head())

,시도명,시군구명,읍면동명,아동인구수,고령자인구수,총인구,외국인
0,서울특별시,종로구,사직동,619,1826,8411,273
1,서울특별시,종로구,삼청동,130,577,2060,66
2,서울특별시,종로구,부암동,683,1953,9057,345
3,서울특별시,종로구,평창동,1484,3854,16392,294
4,서울특별시,종로구,무악동,903,1492,7446,48


### 6. 법정동 코드 merge

In [45]:
# cd 데이터프레임에서 필요한 컬럼만 선택하고 컬럼명 변경
cd_subset = cd[['시도명', '시군구명', '읍면동명', '법정동코드']].copy()
cd_subset = cd_subset.rename(columns={'법정동코드': '행정동코드'})

# total 데이터프레임과 cd_subset 데이터프레임을 병합
# inner merge를 사용하여 total과 cd_subset 모두에 존재하는 키만 남깁니다.
total = pd.merge(total, cd_subset, on=['시도명', '시군구명', '읍면동명'], how='left')

# '행정동코드' 컬럼의 위치를 조정
cols = total.columns.tolist()
# '읍면동명' 컬럼의 인덱스를 찾습니다.
읍면동명_idx = cols.index('읍면동명')
# '행정동코드' 컬럼을 '읍면동명' 컬럼 바로 다음으로 이동시킵니다.
# '행정동코드'가 cols 리스트에 있는지 확인합니다.
if '행정동코드' in cols:
    cols.insert(읍면동명_idx + 1, cols.pop(cols.index('행정동코드')))
    total = total[cols]

display(total.head())

,시도명,시군구명,읍면동명,행정동코드,아동인구수,고령자인구수,총인구,외국인
0,서울특별시,종로구,사직동,1.111012e+09,619,1826,8411,273
1,서울특별시,종로구,삼청동,1.111014e+09,130,577,2060,66
2,서울특별시,종로구,부암동,1.111018e+09,683,1953,9057,345
3,서울특별시,종로구,평창동,1.111018e+09,1484,3854,16392,294
4,서울특별시,종로구,무악동,1.111019e+09,903,1492,7446,48


In [46]:
total.shape

(3554, 8)

## 사회취약계층 계산

## 전처리된 데이터 export

In [49]:
from pathlib import Path

out_dir = Path.cwd().parent / 'data' / 'processed'   # notebooks → .. → data/processed
out_dir.mkdir(parents=True, exist_ok=True)           # 폴더 없으면 생성
total.to_excel(out_dir / '202506_읍면동_사회취약계층표.xlsx', index=False)

In [50]:
total.isnull().sum()

시도명          0
시군구명         0
읍면동명         0
행정동코드     1419
아동인구수        3
고령자인구수       1
총인구          0
외국인          0
dtype: int64